In [1]:
%matplotlib inline
import h5py
import numpy as np
import os
from UG2.utils import data as data_utils
from UG2.utils import image as image_utils
from UG2.config import Config
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import scipy.ndimage as im
import time
from UG2 import main

In [2]:
data_path            = "/data/UG2_data/h5Files/"
traindedModels_path  = "/data/UG2_data/trainedModels/"

In [3]:
## create blurred-dataset 
kernel_blurring_flag= True
num_files           = 10
factor              = 1
patch_size          = np.array([224,224])
epochsList          = [700]
learningRateList    = [0.1]
numberTrainingKernel= [1]
flags               = [0,1]
numberOfExperiments = len(epochsList)*len(learningRateList)
dataset_name        = "imagenet"
blur_Seed           = "FixedSeed"

In [4]:
#os.environ["CUDA_VISIBLE_DEVICES"]="1"
trainStartIndex = 0
numberCompletedExperiments = 0
fileStartIndex  = 1
for i in range(trainStartIndex, len(epochsList)):
    for nLR in learningRateList:
        for nTK in numberTrainingKernel:
            config = Config() 
            config.epochs = epochsList[i]
            config.cuda = True
            config.data_parallel = True
            config.lr = nLR
            config.data_path = data_path
            config.data_files = []
            config.lr_step_list = [10,30]
            config.step_decay_param = 0.2
            for fileIndex in range(fileStartIndex,num_files+1):
                if kernel_blurring_flag:
                    config.data_files.append(os.path.join(data_path, dataset_name+"_nTK_"+str(nTK)+ "_flag_" + "_".join([str(f) for f in flags])+"factor"+str(factor)+"nFile"+str(fileIndex)+blur_Seed+".h5"))
                else:        
                    config.data_files.append(os.path.join(data_path, dataset_name+".h5"))
                    
            config.batch_size = 50
            config.print_step = 1
            config.checkpoint = 1
            config.exit_loss_window = 5
            config.loss_epsilon = 0.01
            config.resume_training_flag = False
            config.kernel_blurring_flag = False
            config.discriminator = "feat_ext"
            config.factor      = factor
            config.img_size    = 64
            config.num_workers = 8
            config.data_format = "h5"
            
            if i:
                config.epochs = epochsList[i]-epochsList[i-1]
                config.resume_training_flag = True
                config.resume_model_path = traindedModels_path
                config.resume_model_name = "modellrRate"+str(nLR).replace(".","_")+"nEpochs"+str(epochsList[i-1])+".pth"

            config.model_path = traindedModels_path
            config.model_name = "modellrRate"+str(nLR).replace(".","_")+"nEpochs"+str(epochsList[i])+"nTK"+str(nTK)+"nFile"+str(num_files)+blur_Seed+".pth"
            numberCompletedExperiments = numberCompletedExperiments+1
            main.train(config)
        
            print("Completed training on file: "+str(num_files))        

time:  2.7647178173065186  Error:  5.2867820859
saved checkpoint at epoch:  0
time:  2.768672466278076  Error:  1.53203002214
saved checkpoint at epoch:  1
time:  2.80387806892395  Error:  1.34624455869
saved checkpoint at epoch:  2
time:  2.812490940093994  Error:  1.2938867867
saved checkpoint at epoch:  3
time:  2.8290436267852783  Error:  1.26323080957
saved checkpoint at epoch:  4
time:  2.8614072799682617  Error:  1.24652545154
saved checkpoint at epoch:  5
time:  2.839906930923462  Error:  1.2372096777
saved checkpoint at epoch:  6
time:  2.787442922592163  Error:  1.23284130991
saved checkpoint at epoch:  7
time:  2.8557825088500977  Error:  1.23350311518
saved checkpoint at epoch:  8
time:  2.8315932750701904  Error:  1.23791031837
saved checkpoint at epoch:  9
time:  2.8360068798065186  Error:  1.24331414998
saved checkpoint at epoch:  10
time:  3.144770622253418  Error:  1.24780044556
saved checkpoint at epoch:  11
time:  3.163313150405884  Error:  1.25049962699
saved checkp

Process Process-6986:
Process Process-6987:
Process Process-6985:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/snagesh/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/snagesh/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/snagesh/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/snagesh/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/snagesh/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/home/snagesh/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/snagesh/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in 

time:  2.8374879360198975  Error:  1.00533919533
saved checkpoint at epoch:  87
Model saved as:  modellrRate0_1nEpochs700nTK1nFile10FixedSeed.pth
Completed training on file: 10
